In [2]:
# This script resfers to https://github.com/evalplus/evalplus
!pip install evalplus --upgrade

In [27]:
!pip install transformers
!pip install bitsandbytes
!pip install peft
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer, PhiForCausalLM
import torch
from peft import LoraConfig, prepare_model_for_kbit_training, AutoPeftModelForCausalLM, PeftModel


In [29]:
###### TODO: replace model to trained phi-2 #####
torch.set_default_device("cuda")

# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

In [32]:
from evalplus.data import get_human_eval_plus, write_jsonl
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, PhiForCausalLM
def GEN_SOLUTION(model, problem):
  inputs = tokenizer(problem, return_tensors="pt", return_attention_mask=False).to('cuda')

  outputs = model.generate(**inputs, max_length=len(problem)+300, pad_token_id=tokenizer.eos_token_id)
  text = tokenizer.batch_decode(outputs)[0]

  return text


In [33]:
# Implement the GEN_SOLUTION function by calling the LLM to produce the complete solution (include the code) 
# and save the samples to samples.jsonl:
# Load the model with 4bit quauntization
bnb_config = BitsAndBytesConfig(
        bnb_4bit_compute_dtype='bfloat16',
        bnb_4bit_quant_type='nf4',
        bnb_4bit_use_double_quant=True,
        load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained('/home/dylanz/group_project/eecs-545-project/trained_models/artifacts', trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
          'microsoft/phi-2', 
          quantization_config=bnb_config,
          trust_remote_code=True
)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, './trained_models/' + 'artifacts')
model = model.merge_and_unload()

samples = [dict(task_id=task_id, solution=GEN_SOLUTION(model, problem["prompt"]))
    for task_id, problem in get_human_eval_plus().items()]
write_jsonl("samples.jsonl", samples)  ### write the generate response to local

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
!git clone https://github.com/evalplus/evalplus.git

Cloning into 'evalplus'...
remote: Enumerating objects: 6332, done.
remote: Counting objects: 100% (6329/6329), done.
remote: Compressing objects: 100% (2206/2206), done.
remote: Total 6332 (delta 3962), reused 6107 (delta 3878), pack-reused 3
Receiving objects: 100% (6332/6332), 5.18 MiB | 11.15 MiB/s, done.
Resolving deltas: 100% (3962/3962), done.


In [ ]:
!cd ./evalplus/evalplus

In [ ]:
# LLM-generated text may not be compilable code for including natural language lines or incomplete extra code.
# We provide a tool namely evalplus.sanitize to clean up the code:
!python ./evalplus/sanitize.py --samples ./samples.jsonl

In [ ]:
# Checking the compilability of post-processed code:
!python evalplus/syncheck.py --samples ./samples-sanitized.jsonl --dataset humaneval